In [1]:
from bio import Animal, AnimalDraw
from ipywidgets import interact


def animal_maker(seed, length, gradation, straightness, distribution, leg_count, ground):
    global DRAW
    animal = Animal.from_params(length, gradation / 10, straightness / 100, distribution / 10, min(leg_count, length - 1), seed)
    DRAW = AnimalDraw(animal)
    DRAW.set_ground_level(ground)
    return DRAW.draw(150)


interact(animal_maker, seed=(1, 999999), length=(7, 15),
         gradation=(2.0, 500.), straightness=(1.0, 500.0), distribution=(50.0, 500.0), leg_count=(1, 4),
         ground=(0, 300))


interactive(children=(IntSlider(value=500000, description='seed', max=999999, min=1), IntSlider(value=11, desc…

<function __main__.animal_maker(seed, length, gradation, straightness, distribution, leg_count, ground)>

In [6]:
DRAW.save_3d('test.stl')